<a href="https://colab.research.google.com/github/ab3131/NBAInjuryPredictor/blob/main/NBAInjuryPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Statements

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Data Cleaning

In [3]:
player_stats = pd.read_csv('player_stats.csv')
injuries_original = pd.read_csv('NBA Player Injury Stats(1951 - 2023).csv')

In [4]:
player_stats.columns[0]


'Unnamed: 0'

In [5]:
injuries_original.columns[0]

'Unnamed: 0'

In [6]:
player_stats = player_stats.drop(columns = player_stats.columns[0], axis=1)

In [7]:
player_stats

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996-97
1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,34,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996-97
2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,12,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996-97
3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,7,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996-97
4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,22,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996-97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,Joel Embiid,PHI,29.0,213.36,127.005760,Kansas,Cameroon,2014,1,3,...,33.1,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233,2022-23
12840,John Butler Jr.,POR,20.0,213.36,86.182480,Florida State,USA,Undrafted,Undrafted,Undrafted,...,2.4,0.9,0.6,-16.1,0.012,0.065,0.102,0.411,0.066,2022-23
12841,John Collins,ATL,25.0,205.74,102.511792,Wake Forest,USA,2017,1,19,...,13.1,6.5,1.2,-0.2,0.035,0.180,0.168,0.593,0.052,2022-23
12842,Jericho Sims,NYK,24.0,208.28,113.398000,Texas,USA,2021,2,58,...,3.4,4.7,0.5,-6.7,0.117,0.175,0.074,0.780,0.044,2022-23


In [8]:
injuries_original = injuries_original.drop(columns=injuries_original.columns[0], axis=1)

In [9]:
injuries_original

,Date,Team,Acquired,Relinquished,Notes
0,1951-12-25,Bullets,NaN,Don Barksdale,placed on IL
1,1952-12-26,Knicks,NaN,Max Zaslofsky,placed on IL with torn side muscle
2,1956-12-29,Knicks,NaN,Jim Baechtold,placed on inactive list
3,1959-01-16,Lakers,NaN,Elgin Baylor,player refused to play after being denied a ro...
4,1961-11-26,Lakers,NaN,Elgin Baylor,player reported for military duty
...,...,...,...,...,...
37662,2023-04-16,Clippers,Marcus Morris,NaN,activated from IL
37663,2023-04-16,Grizzlies,Dillon Brooks,NaN,activated from IL
37664,2023-04-16,Grizzlies,Ja Morant,NaN,activated from IL
37665,2023-04-16,Grizzlies,Jaren Jackson Jr.,NaN,activated from IL


Filter out injuries prior to 1996, where the player_stats dataset begins. We also don't want any rows where someone was activated, we care about the injury itself

In [10]:
injuries_date_filtered = injuries_original[(injuries_original['Date']>'1996-07-01') & (injuries_original['Date']<'2023-07-01')]

In [11]:
injuries_date_filtered

,Date,Team,Acquired,Relinquished,Notes
2055,1996-09-11,Pacers,NaN,Rik Smits,placed on IR with surgery on feet (date approx...
2056,1996-10-30,Blazers,NaN,Aleksandar Djordjevic,placed on IL with sprained right ankle
2057,1996-10-30,Blazers,NaN,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
2058,1996-10-30,Blazers,NaN,Marcus Brown,placed on IL with strained right groin
2059,1996-10-30,Knicks,NaN,Chris Childs,placed on IR with broken right leg
...,...,...,...,...,...
37662,2023-04-16,Clippers,Marcus Morris,NaN,activated from IL
37663,2023-04-16,Grizzlies,Dillon Brooks,NaN,activated from IL
37664,2023-04-16,Grizzlies,Ja Morant,NaN,activated from IL
37665,2023-04-16,Grizzlies,Jaren Jackson Jr.,NaN,activated from IL


In [12]:
injuries = injuries_date_filtered[~(injuries_date_filtered['Relinquished'].isna())]

In [13]:
injuries

,Date,Team,Acquired,Relinquished,Notes
2055,1996-09-11,Pacers,NaN,Rik Smits,placed on IR with surgery on feet (date approx...
2056,1996-10-30,Blazers,NaN,Aleksandar Djordjevic,placed on IL with sprained right ankle
2057,1996-10-30,Blazers,NaN,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
2058,1996-10-30,Blazers,NaN,Marcus Brown,placed on IL with strained right groin
2059,1996-10-30,Knicks,NaN,Chris Childs,placed on IR with broken right leg
...,...,...,...,...,...
37624,2023-04-09,Grizzlies,NaN,Ja Morant,placed on IL with right hand injury
37625,2023-04-09,Grizzlies,NaN,Jaren Jackson Jr.,placed on IL with sore left elbow
37630,2023-04-09,Kings,NaN,Davion Mitchell,placed on IL with sore left knee
37631,2023-04-09,Knicks,NaN,Trevor Keels,placed on IL with illness


In [14]:
injuries = injuries.drop(columns = 'Acquired')

When combining, have to keep date(convert to season?), team, relinquished, and notes

In [15]:
injuries = injuries.reset_index(drop=True)

In [16]:
injuries

,Date,Team,Relinquished,Notes
0,1996-09-11,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996-10-30,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
2,1996-10-30,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
3,1996-10-30,Blazers,Marcus Brown,placed on IL with strained right groin
4,1996-10-30,Knicks,Chris Childs,placed on IR with broken right leg
...,...,...,...,...
18887,2023-04-09,Grizzlies,Ja Morant,placed on IL with right hand injury
18888,2023-04-09,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow
18889,2023-04-09,Kings,Davion Mitchell,placed on IL with sore left knee
18890,2023-04-09,Knicks,Trevor Keels,placed on IL with illness


Separate stats by season for categorization(Season starts on October 12th, ends October 11th of next year)

In [17]:
def find_season(date):
  year = int(date[0:4])
  month = int(date[5:7])
  day = int(date[8:10])
  if month<10:
    return year-1
  elif month==10:
    if day<12:
      return year-1
    else:
      return year
  else:
    return year

In [18]:
injuries['Date'] = injuries['Date'].apply(find_season)

In [19]:
injuries

,Date,Team,Relinquished,Notes
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
2,1996,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
3,1996,Blazers,Marcus Brown,placed on IL with strained right groin
4,1996,Knicks,Chris Childs,placed on IR with broken right leg
...,...,...,...,...
18887,2022,Grizzlies,Ja Morant,placed on IL with right hand injury
18888,2022,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow
18889,2022,Kings,Davion Mitchell,placed on IL with sore left knee
18890,2022,Knicks,Trevor Keels,placed on IL with illness


In [20]:
def one_year(season):
  return int(season[0:4])

In [21]:
player_stats['season'] = player_stats['season'].apply(one_year)

In [22]:
player_stats

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,Randy Livingston,HOU,22.0,193.04,94.800728,Louisiana State,USA,1996,2,42,...,3.9,1.5,2.4,0.3,0.042,0.071,0.169,0.487,0.248,1996
1,Gaylon Nickerson,WAS,28.0,190.50,86.182480,Northwestern Oklahoma,USA,1994,2,34,...,3.8,1.3,0.3,8.9,0.030,0.111,0.174,0.497,0.043,1996
2,George Lynch,VAN,26.0,203.20,103.418976,North Carolina,USA,1993,1,12,...,8.3,6.4,1.9,-8.2,0.106,0.185,0.175,0.512,0.125,1996
3,George McCloud,LAL,30.0,203.20,102.058200,Florida State,USA,1989,1,7,...,10.2,2.8,1.7,-2.7,0.027,0.111,0.206,0.527,0.125,1996
4,George Zidek,DEN,23.0,213.36,119.748288,UCLA,USA,1995,1,22,...,2.8,1.7,0.3,-14.1,0.102,0.169,0.195,0.500,0.064,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,Joel Embiid,PHI,29.0,213.36,127.005760,Kansas,Cameroon,2014,1,3,...,33.1,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233,2022
12840,John Butler Jr.,POR,20.0,213.36,86.182480,Florida State,USA,Undrafted,Undrafted,Undrafted,...,2.4,0.9,0.6,-16.1,0.012,0.065,0.102,0.411,0.066,2022
12841,John Collins,ATL,25.0,205.74,102.511792,Wake Forest,USA,2017,1,19,...,13.1,6.5,1.2,-0.2,0.035,0.180,0.168,0.593,0.052,2022
12842,Jericho Sims,NYK,24.0,208.28,113.398000,Texas,USA,2021,2,58,...,3.4,4.7,0.5,-6.7,0.117,0.175,0.074,0.780,0.044,2022


In [23]:
injuries = injuries.rename(columns={"Relinquished": "player_name"})
injuries = injuries.rename(columns={"Date": "season"})

In [24]:
injuries

,season,Team,player_name,Notes
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
2,1996,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...
3,1996,Blazers,Marcus Brown,placed on IL with strained right groin
4,1996,Knicks,Chris Childs,placed on IR with broken right leg
...,...,...,...,...
18887,2022,Grizzlies,Ja Morant,placed on IL with right hand injury
18888,2022,Grizzlies,Jaren Jackson Jr.,placed on IL with sore left elbow
18889,2022,Kings,Davion Mitchell,placed on IL with sore left knee
18890,2022,Knicks,Trevor Keels,placed on IL with illness


In [25]:
player_stats['season'] = player_stats['season'].astype(int)
injuries['season'] = injuries['season'].astype(int)
player_stats['player_name'] = player_stats['player_name'].astype(str)
injuries['player_name'] = injuries['player_name'].astype(str)


In [26]:
final = pd.merge( injuries, player_stats, how='outer', left_on=['season', 'player_name'], right_on = ['season', 'player_name'])

In [27]:
final

,season,Team,player_name,Notes,team_abbreviation,age,player_height,player_weight,college,country,...,gp,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996,Blazers,Aleksandar Djordjevic,placed on IL with flu,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996,Blazers,Jermaine O'Neal,placed on IL with sprained left knee / bone co...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996,Blazers,Marcus Brown,placed on IL with strained right groin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31731,2022,NaN,Joel Embiid,NaN,PHI,29.0,213.36,127.005760,Kansas,Cameroon,...,66.0,33.1,10.2,4.2,8.8,0.057,0.243,0.370,0.655,0.233
31732,2022,NaN,John Butler Jr.,NaN,POR,20.0,213.36,86.182480,Florida State,USA,...,19.0,2.4,0.9,0.6,-16.1,0.012,0.065,0.102,0.411,0.066
31733,2022,NaN,John Collins,NaN,ATL,25.0,205.74,102.511792,Wake Forest,USA,...,71.0,13.1,6.5,1.2,-0.2,0.035,0.180,0.168,0.593,0.052
31734,2022,NaN,Jericho Sims,NaN,NYK,24.0,208.28,113.398000,Texas,USA,...,52.0,3.4,4.7,0.5,-6.7,0.117,0.175,0.074,0.780,0.044


pd.merge not giving ideal values, so we will fix this

In [28]:
injuries = injuries.sort_values(by=['season', 'player_name'])

In [29]:
injuries = injuries.reset_index(drop=True)

In [30]:
injuries

,season,Team,player_name,Notes
0,1995,Pacers,Rik Smits,placed on IR with surgery on feet (date approx...
1,1996,Jazz,Adam Keefe,placed on IR with bruised left hamstring
2,1996,76ers,Adrian Caldwell,placed on IR with bursitis in right foot
3,1996,Hawks,Alan Henderson,placed on IR with acute viral pancreatitis / g...
4,1996,Blazers,Aleksandar Djordjevic,placed on IL with sprained right ankle
...,...,...,...,...
18887,2022,Pelicans,Zion Williamson,placed on IL with bruised right foot
18888,2022,Pelicans,Zion Williamson,placed on IL with NBA health and safety protocols
18889,2022,Pelicans,Zion Williamson,placed on IL with strained right hamstring (ou...
18890,2022,Magic,left knee injury (DTD),placed on IL with left knee injury


In [31]:
injuries_lists = injuries.copy()
listtodrop = []
startindex = 1
for i in range(1, len(injuries)):
  injuries_lists['Notes'][i] = [injuries['Notes'][i]]
  if injuries['player_name'][i] == injuries['player_name'][i-1]:
    listtodrop.append(i)
    injuries_lists['Notes'][startindex].append(injuries['Notes'][i])
  else:
    startindex = i
injuries_lists = injuries_lists.drop(listtodrop, axis='index')

Streaming output truncated to the last 5000 lines.
<ipython-input-31-fea5ea898190>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injuries_lists['Notes'][i] = [injuries['Notes'][i]]
<ipython-input-31-fea5ea898190>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injuries_lists['Notes'][i] = [injuries['Notes'][i]]
<ipython-input-31-fea5ea898190>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injuries_lists['Notes'][i] = [injuries['N

In [32]:
injuries_lists['Notes'][0] = [injuries['Notes'][0]]

<ipython-input-32-b782bbbe071c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injuries_lists['Notes'][0] = [injuries['Notes'][0]]


In [33]:
injuries_lists

,season,Team,player_name,Notes
0,1995,Pacers,Rik Smits,[placed on IR with surgery on feet (date appro...
1,1996,Jazz,Adam Keefe,[placed on IR with bruised left hamstring]
2,1996,76ers,Adrian Caldwell,[placed on IR with bursitis in right foot]
3,1996,Hawks,Alan Henderson,[placed on IR with acute viral pancreatitis / ...
4,1996,Blazers,Aleksandar Djordjevic,"[placed on IL with sprained right ankle, place..."
...,...,...,...,...
18883,2022,Nuggets,Zeke Nnaji,"[placed on IL with sprained right shoulder, pl..."
18885,2022,Grizzlies,Ziaire Williams,[placed on IL with sore right foot]
18886,2022,Pelicans,Zion Williamson,"[placed on IL with bruised lower back, placed ..."
18890,2022,Magic,left knee injury (DTD),[placed on IL with left knee injury]


In [34]:
injuries_lists['Notes']

0        [placed on IR with surgery on feet (date appro...
1               [placed on IR with bruised left hamstring]
2               [placed on IR with bursitis in right foot]
3        [placed on IR with acute viral pancreatitis / ...
4        [placed on IL with sprained right ankle, place...
                               ...                        
18883    [placed on IL with sprained right shoulder, pl...
18885                  [placed on IL with sore right foot]
18886    [placed on IL with bruised lower back, placed ...
18890                 [placed on IL with left knee injury]
18891         [placed on IL with strained left quadriceps]
Name: Notes, Length: 8077, dtype: object

In [35]:
player_stats = player_stats.sort_values(by = ['season', 'player_name'])

In [36]:
player_stats = player_stats.reset_index(drop=True)

In [37]:
player_stats

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,pts,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season
0,A.C. Green,DAL,33.0,205.74,102.058200,Oregon State,USA,1985,1,23,...,7.2,7.9,0.8,-8.0,0.100,0.207,0.119,0.523,0.045,1996
1,Aaron McKie,DET,24.0,195.58,94.800728,Temple,USA,1994,1,17,...,5.2,2.7,1.9,5.2,0.031,0.129,0.147,0.524,0.163,1996
2,Aaron Williams,VAN,25.0,205.74,102.058200,Xavier,USA,Undrafted,Undrafted,Undrafted,...,6.2,4.3,0.5,-8.3,0.129,0.163,0.168,0.599,0.051,1996
3,Acie Earl,MIL,27.0,210.82,108.862080,Iowa,USA,1993,1,19,...,4.0,2.0,0.4,-6.4,0.079,0.140,0.224,0.435,0.077,1996
4,Adam Keefe,UTA,27.0,205.74,109.315672,Stanford,USA,1992,1,10,...,3.8,3.5,0.5,6.7,0.110,0.177,0.126,0.572,0.051,1996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,Zach Collins,SAS,25.0,210.82,113.398000,Gonzaga,USA,2017,1,10,...,11.6,6.4,2.9,-7.5,0.076,0.190,0.209,0.599,0.180,2022
12840,Zach LaVine,CHI,28.0,195.58,90.718400,UCLA,USA,2014,1,13,...,24.8,4.5,4.2,0.3,0.016,0.108,0.278,0.607,0.187,2022
12841,Zeke Nnaji,DEN,22.0,205.74,108.862080,Arizona,USA,2020,1,22,...,5.2,2.6,0.3,-5.9,0.087,0.099,0.149,0.620,0.040,2022
12842,Ziaire Williams,MEM,21.0,205.74,83.914520,Stanford,USA,2021,1,10,...,5.7,2.1,0.9,-5.2,0.028,0.105,0.178,0.511,0.086,2022


In [38]:
player_stats['injuries'] = [[] for i in range(len(player_stats))]

In [39]:
player_stats

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,injuries
0,A.C. Green,DAL,33.0,205.74,102.058200,Oregon State,USA,1985,1,23,...,7.9,0.8,-8.0,0.100,0.207,0.119,0.523,0.045,1996,[]
1,Aaron McKie,DET,24.0,195.58,94.800728,Temple,USA,1994,1,17,...,2.7,1.9,5.2,0.031,0.129,0.147,0.524,0.163,1996,[]
2,Aaron Williams,VAN,25.0,205.74,102.058200,Xavier,USA,Undrafted,Undrafted,Undrafted,...,4.3,0.5,-8.3,0.129,0.163,0.168,0.599,0.051,1996,[]
3,Acie Earl,MIL,27.0,210.82,108.862080,Iowa,USA,1993,1,19,...,2.0,0.4,-6.4,0.079,0.140,0.224,0.435,0.077,1996,[]
4,Adam Keefe,UTA,27.0,205.74,109.315672,Stanford,USA,1992,1,10,...,3.5,0.5,6.7,0.110,0.177,0.126,0.572,0.051,1996,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,Zach Collins,SAS,25.0,210.82,113.398000,Gonzaga,USA,2017,1,10,...,6.4,2.9,-7.5,0.076,0.190,0.209,0.599,0.180,2022,[]
12840,Zach LaVine,CHI,28.0,195.58,90.718400,UCLA,USA,2014,1,13,...,4.5,4.2,0.3,0.016,0.108,0.278,0.607,0.187,2022,[]
12841,Zeke Nnaji,DEN,22.0,205.74,108.862080,Arizona,USA,2020,1,22,...,2.6,0.3,-5.9,0.087,0.099,0.149,0.620,0.040,2022,[]
12842,Ziaire Williams,MEM,21.0,205.74,83.914520,Stanford,USA,2021,1,10,...,2.1,0.9,-5.2,0.028,0.105,0.178,0.511,0.086,2022,[]


In [40]:
injuries_lists

,season,Team,player_name,Notes
0,1995,Pacers,Rik Smits,[placed on IR with surgery on feet (date appro...
1,1996,Jazz,Adam Keefe,[placed on IR with bruised left hamstring]
2,1996,76ers,Adrian Caldwell,[placed on IR with bursitis in right foot]
3,1996,Hawks,Alan Henderson,[placed on IR with acute viral pancreatitis / ...
4,1996,Blazers,Aleksandar Djordjevic,"[placed on IL with sprained right ankle, place..."
...,...,...,...,...
18883,2022,Nuggets,Zeke Nnaji,"[placed on IL with sprained right shoulder, pl..."
18885,2022,Grizzlies,Ziaire Williams,[placed on IL with sore right foot]
18886,2022,Pelicans,Zion Williamson,"[placed on IL with bruised lower back, placed ..."
18890,2022,Magic,left knee injury (DTD),[placed on IL with left knee injury]


In [67]:
player_stats

,player_name,team_abbreviation,age,player_height,player_weight,college,country,draft_year,draft_round,draft_number,...,reb,ast,net_rating,oreb_pct,dreb_pct,usg_pct,ts_pct,ast_pct,season,injuries
0,A.C. Green,DAL,33.0,205.74,102.058200,Oregon State,USA,1985,1,23,...,7.9,0.8,-8.0,0.100,0.207,0.119,0.523,0.045,1996,[]
1,Aaron McKie,DET,24.0,195.58,94.800728,Temple,USA,1994,1,17,...,2.7,1.9,5.2,0.031,0.129,0.147,0.524,0.163,1996,[]
2,Aaron Williams,VAN,25.0,205.74,102.058200,Xavier,USA,Undrafted,Undrafted,Undrafted,...,4.3,0.5,-8.3,0.129,0.163,0.168,0.599,0.051,1996,[]
3,Acie Earl,MIL,27.0,210.82,108.862080,Iowa,USA,1993,1,19,...,2.0,0.4,-6.4,0.079,0.140,0.224,0.435,0.077,1996,[]
4,Adam Keefe,UTA,27.0,205.74,109.315672,Stanford,USA,1992,1,10,...,3.5,0.5,6.7,0.110,0.177,0.126,0.572,0.051,1996,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12839,Zach Collins,SAS,25.0,210.82,113.398000,Gonzaga,USA,2017,1,10,...,6.4,2.9,-7.5,0.076,0.190,0.209,0.599,0.180,2022,[]
12840,Zach LaVine,CHI,28.0,195.58,90.718400,UCLA,USA,2014,1,13,...,4.5,4.2,0.3,0.016,0.108,0.278,0.607,0.187,2022,[]
12841,Zeke Nnaji,DEN,22.0,205.74,108.862080,Arizona,USA,2020,1,22,...,2.6,0.3,-5.9,0.087,0.099,0.149,0.620,0.040,2022,[]
12842,Ziaire Williams,MEM,21.0,205.74,83.914520,Stanford,USA,2021,1,10,...,2.1,0.9,-5.2,0.028,0.105,0.178,0.511,0.086,2022,[]


In [64]:
injuries_lists = injuries_lists.reset_index(drop=True)

In [71]:
injuries_lists

,season,Team,player_name,Notes
0,1995,Pacers,Rik Smits,[placed on IR with surgery on feet (date appro...
1,1996,Jazz,Adam Keefe,[placed on IR with bruised left hamstring]
2,1996,76ers,Adrian Caldwell,[placed on IR with bursitis in right foot]
3,1996,Hawks,Alan Henderson,[placed on IR with acute viral pancreatitis / ...
4,1996,Blazers,Aleksandar Djordjevic,"[placed on IL with sprained right ankle, place..."
...,...,...,...,...
8072,2022,Nuggets,Zeke Nnaji,"[placed on IL with sprained right shoulder, pl..."
8073,2022,Grizzlies,Ziaire Williams,[placed on IL with sore right foot]
8074,2022,Pelicans,Zion Williamson,"[placed on IL with bruised lower back, placed ..."
8075,2022,Magic,left knee injury (DTD),[placed on IL with left knee injury]


In [84]:
for i in range(len(injuries_lists)):
  injuries_lists['player_name'][i] = injuries_lists['player_name'][i][1: len(injuries_lists['player_name'][i])]

Streaming output truncated to the last 5000 lines.
<ipython-input-84-aa281b22cd6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injuries_lists['player_name'][i] = injuries_lists['player_name'][i][1: len(injuries_lists['player_name'][i])]
<ipython-input-84-aa281b22cd6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  injuries_lists['player_name'][i] = injuries_lists['player_name'][i][1: len(injuries_lists['player_name'][i])]
<ipython-input-84-aa281b22cd6b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

In [86]:
player_index= 0
for i in range(1, len(injuries_lists)):
  while player_index<len(player_stats) and player_stats['player_name'][player_index]<=injuries_lists['player_name'][i] and player_stats['season'][player_index]<=injuries_lists['season'][i]:

    if player_stats['player_name'][player_index]==injuries_lists['player_name'][i] and player_stats['season'][player_index]==injuries_lists['season'][i]:
      player_stats['injuries'][player_index] = injuries_lists['Notes'][i]

      player_index+=1
      break
    player_index+=1


Streaming output truncated to the last 5000 lines.
<ipython-input-86-a77b546da4b6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats['injuries'][player_index] = injuries_lists['Notes'][i]
<ipython-input-86-a77b546da4b6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats['injuries'][player_index] = injuries_lists['Notes'][i]
<ipython-input-86-a77b546da4b6>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_st

In [88]:
player_stats['injuries'][12843]

['placed on IL with bruised lower back',
 'placed on IL with bruised right foot',
 'placed on IL with NBA health and safety protocols',
 'placed on IL with strained right hamstring (out for season)']

In [92]:
player_stats[player_stats['player_name']=='Zion Williamson']['gp']

11158    24
11699    61
12843    29
Name: gp, dtype: int64